## Лабораторная работа 9: Использование разработанного пайплайна для многомерной регрессии
Суть: Расширение пайплайна для работы с задачами многомерной регрессии. Добавление поддержки нескольких выходных переменных.

Многомерные данные: Набор данных с несколькими входными и выходными переменными.

Матрица весов: Математическое представление влияния входных данных на выходные.

Совместное предсказание: Одновременное предсказание нескольких выходных значений.

1.1 Подключение библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

In [2]:
uploaded = files.upload()

Saving 50_Startups.csv to 50_Startups.csv


1.2 Загрузка данных и разделение на матрицу признаков и зависимую переменную

In [3]:
dataset = pd.read_csv('50_Startups.csv')
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [4]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values
print("Матрица признаков"); print(X[:5])
print("Зависимая переменная"); print(y[:5])

Матрица признаков
[[165349.2 136897.8 471784.1 'New York']
 [162597.7 151377.59 443898.53 'California']
 [153441.51 101145.55 407934.54 'Florida']
 [144372.41 118671.85 383199.62 'New York']
 [142107.34 91391.77 366168.42 'Florida']]
Зависимая переменная
[192261.83 191792.06 191050.39 182901.99 166187.94]


1.3 Обработка пропущенных значений

In [5]:
from sklearn.impute import SimpleImputer
# Создание объекта SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# Применение SimpleImputer для заполнения пропусков
X[:, 1:3] = imputer.fit_transform(X[:, 1:3])
# Печать обработанного массива X
print(X)

[[165349.2 136897.8 471784.1 'New York']
 [162597.7 151377.59 443898.53 'California']
 [153441.51 101145.55 407934.54 'Florida']
 [144372.41 118671.85 383199.62 'New York']
 [142107.34 91391.77 366168.42 'Florida']
 [131876.9 99814.71 362861.36 'New York']
 [134615.46 147198.87 127716.82 'California']
 [130298.13 145530.06 323876.68 'Florida']
 [120542.52 148718.95 311613.29 'New York']
 [123334.88 108679.17 304981.62 'California']
 [101913.08 110594.11 229160.95 'Florida']
 [100671.96 91790.61 249744.55 'California']
 [93863.75 127320.38 249839.44 'Florida']
 [91992.39 135495.07 252664.93 'California']
 [119943.24 156547.42 256512.92 'Florida']
 [114523.61 122616.84 261776.23 'New York']
 [78013.11 121597.55 264346.06 'California']
 [94657.16 145077.58 282574.31 'New York']
 [91749.16 114175.79 294919.57 'Florida']
 [86419.7 153514.11 0.0 'New York']
 [76253.86 113867.3 298664.47 'California']
 [78389.47 153773.43 299737.29 'New York']
 [73994.56 122782.75 303319.26 'Florida']
 [67532

1.4 Обработка категориальных данных

1.4.1 Замена категории кодом (LabelEncoder)

In [6]:
from sklearn.preprocessing import LabelEncoder
Labelencoder_y = LabelEncoder()
print("Зависимая переменная до обработки")
print(y)
print(y)
y = Labelencoder_y.fit_transform(y)
print("Зависимая переменная после обработки")

Зависимая переменная до обработки
[192261.83 191792.06 191050.39 182901.99 166187.94 156991.12 156122.51
 155752.6  152211.77 149759.96 146121.95 144259.4  141585.52 134307.35
 132602.65 129917.04 126992.93 125370.37 124266.9  122776.86 118474.03
 111313.02 110352.25 108733.99 108552.04 107404.34 105733.54 105008.31
 103282.38 101004.64  99937.59  97483.56  97427.84  96778.92  96712.8
  96479.51  90708.19  89949.14  81229.06  81005.76  78239.91  77798.83
  71498.49  69758.98  65200.33  64926.08  49490.75  42559.73  35673.41
  14681.4 ]
[192261.83 191792.06 191050.39 182901.99 166187.94 156991.12 156122.51
 155752.6  152211.77 149759.96 146121.95 144259.4  141585.52 134307.35
 132602.65 129917.04 126992.93 125370.37 124266.9  122776.86 118474.03
 111313.02 110352.25 108733.99 108552.04 107404.34 105733.54 105008.31
 103282.38 101004.64  99937.59  97483.56  97427.84  96778.92  96712.8
  96479.51  90708.19  89949.14  81229.06  81005.76  78239.91  77798.83
  71498.49  69758.98  65200.33  6

1.4.2 Применение OneHotEncoder

In [7]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
# Преобразование столбца с помощью LabelEncoder
labelencoder = LabelEncoder()
X[:, 3] = labelencoder.fit_transform(X[:, 3])
# Преобразование категориального признака с помощью OneHotEncoder
column_transformer = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(), [3])  # Указываем индекс столбца для кодирования
    ],
    remainder='passthrough'  # Остальные столбцы оставляем без изменений
)
# Применение преобразования
X = column_transformer.fit_transform(X)
# Преобразование в массив
X = np.array(X)
# Вывод результата
print("Перекодировка категориального признака")
print(X[:4, :])

Перекодировка категориального признака
[[0.0 0.0 1.0 165349.2 136897.8 471784.1]
 [1.0 0.0 0.0 162597.7 151377.59 443898.53]
 [0.0 1.0 0.0 153441.51 101145.55 407934.54]
 [0.0 0.0 1.0 144372.41 118671.85 383199.62]]


In [8]:
X = X[:, 1:]
print(X[:4,:])

[[0.0 1.0 165349.2 136897.8 471784.1]
 [0.0 0.0 162597.7 151377.59 443898.53]
 [1.0 0.0 153441.51 101145.55 407934.54]
 [0.0 1.0 144372.41 118671.85 383199.62]]


1.6 Разделение выборки на тестовую и тренировочную

In [9]:
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

1.7 Обучение линейной модели регрессии

In [10]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

1.8 Обработка результатов, тюнинг модели

1.8.1 Предсказание

In [11]:
y_pred = regressor.predict(X_test)
print(y_pred)

[22.73971792 31.79851398 32.76702713 10.17872713 48.95244293 25.07783798
  7.62074212 20.43498693 25.32328448 45.16861644]


1.8.2 Оптимизация модели

In [12]:
import statsmodels.api as sm
# Преобразование данных в тип float
X = X.astype(float)
# Добавляем единичный столбец для константы
X = np.append(arr=np.ones((50, 1)).astype(int), values=X, axis=1)
# Указание доступных индексов
X_opt = X[:, [0, 1, 2, 3, 4, 5]]  # Обновите индексы, чтобы они находились в пределах диапазона
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                     212.6
Date:                Fri, 13 Dec 2024   Prob (F-statistic):           1.22e-29
Time:                        11:49:09   Log-Likelihood:                -123.78
No. Observations:                  50   AIC:                             259.6
Df Residuals:                      44   BIC:                             271.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2027      2.237      0.538      0.594      -3.306       5.711
x1             0.1929      1.095      0.176      0.861      -2.015       2.400
x2            -0.2364      1.058     -0.223      0.824      -2.369       1.896
x3             0.0003   1.51e-05     19.988      0.000       0.000       0.000
x4          5.233e-07    1.7e-05      0.031      0.976   -3.37e-05    3.47e-05
x5          4.897e-06   5.57e-06      0.879      0.384   -6.33e-06    1.61e-05
==============================================================================
Omnibus:                        3.294   Durbin-Watson:                   1.285
Prob(Omnibus):                  0.193   Jarque-Bera (JB):                3.155
Skew:                           0.582   Prob(JB):                        0.206
Kurtosis:                       2.602   Cond. No.                     1.45e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
X_opt = X[:, [0, 1, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                     271.5
Date:                Fri, 13 Dec 2024   Prob (F-statistic):           7.07e-31
Time:                        11:49:18   Log-Likelihood:                -123.81
No. Observations:                  50   AIC:                             257.6
Df Residuals:                      45   BIC:                             267.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0949      2.161      0.507      0.615      -3.258       5.448
x1             0.3135      0.943      0.332      0.741      -1.586       2.213
x2             0.0003   1.49e-05     20.236      0.000       0.000       0.000
x3          5.737e-07   1.68e-05      0.034      0.973   -3.32e-05    3.44e-05
x4          4.897e-06   5.51e-06      0.889      0.379    -6.2e-06     1.6e-05
==============================================================================
Omnibus:                        3.623   Durbin-Watson:                   1.273
Prob(Omnibus):                  0.163   Jarque-Bera (JB):                3.460
Skew:                           0.623   Prob(JB):                        0.177
Kurtosis:                       2.669   Cond. No.                     1.40e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
X_opt = X[:, [0, 3, 4, 5]]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.960
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                     369.1
Date:                Fri, 13 Dec 2024   Prob (F-statistic):           3.56e-32
Time:                        11:49:36   Log-Likelihood:                -123.87
No. Observations:                  50   AIC:                             255.7
Df Residuals:                      46   BIC:                             263.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1176      2.139      0.522      0.604      -3.188       5.423
x1             0.0003   1.47e-05     20.469      0.000       0.000       0.000
x2          8.292e-07   1.66e-05      0.050      0.960   -3.26e-05    3.43e-05
x3           5.25e-06   5.35e-06      0.981      0.332   -5.53e-06     1.6e-05
==============================================================================
Omnibus:                        3.506   Durbin-Watson:                   1.263
Prob(Omnibus):                  0.173   Jarque-Bera (JB):                3.330
Skew:                           0.613   Prob(JB):                        0.189
Kurtosis:                       2.692   Cond. No.                     1.40e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
X_opt = X[:, [0, 3]]
regressor_OLS = sm.OLS(endog = y, exog = X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                     1129.
Date:                Fri, 13 Dec 2024   Prob (F-statistic):           5.25e-35
Time:                        11:49:46   Log-Likelihood:                -124.43
No. Observations:                  50   AIC:                             252.9
Df Residuals:                      48   BIC:                             256.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5703      0.802      1.959      0.056      -0.042       3.182
x1             0.0003   9.26e-06     33.599      0.000       0.000       0.000
==============================================================================
Omnibus:                        3.750   Durbin-Watson:                   1.294
Prob(Omnibus):                  0.153   Jarque-Bera (JB):                3.598
Skew:                           0.633   Prob(JB):                        0.165
Kurtosis:                       2.648   Cond. No.                     1.65e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Индивидуальное задание

In [16]:
uploaded = files.upload()

Saving WineQT.csv to WineQT.csv


In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Чтение данных
dataset = pd.read_csv('WineQT.csv')
print("Данные:")
print(dataset.head())

# Удаляем пропущенные значения, если они есть
dataset = dataset.dropna()

# Определяем матрицу признаков (X) и целевую переменную (y)
# Признаки: Выберите основные параметры, например, 'fixed acidity', 'volatile acidity', 'alcohol'
# Целевая переменная: 'quality'
X = dataset[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'alcohol']].values
y = dataset['quality'].values

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Обучение линейной регрессии
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = regressor.predict(X_test)
print("Предсказания:")
print(y_pred[:5])

# Оптимизация модели с помощью backward elimination
# Добавляем столбец единиц для константы
X = np.append(arr=np.ones((X.shape[0], 1)).astype(int), values=X, axis=1)

# Начальная модель со всеми признаками
X_opt = X[:, [0, 1, 2, 3, 4, 5]]  # Индексы всех признаков
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
print("Модель с полным набором признаков:")
print(regressor_OLS.summary())

# Удаляем признак с наибольшим p-значением и повторяем
X_opt = X[:, [0, 1, 2, 3, 5]]  # Пример: Удален 'citric acid'
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
print("Модель после первого шага:")
print(regressor_OLS.summary())

# Еще одно удаление (по необходимости)
X_opt = X[:, [0, 1, 3, 5]]  # Пример: Удален 'volatile acidity'
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
print("Модель после второго шага:")
print(regressor_OLS.summary())


Данные:
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  Id  
0      9.4        5   0  
1      9.8        5   1  


Вывод:
Модель на полном наборе признаков объясняет 34.2% вариации качества вина.
После удаления признака 'residual sugar' (первый шаг) модель немного ухудшилась (R² = 0.341), но значимость других признаков осталась.
После второго шага, когда удалили 'volatile acidity', модель значительно ухудшилась (R² = 0.273), что показало важность этого признака.
В итоге, наилучшая модель включает 'fixed acidity', 'citric acid', и 'alcohol', но объясняет только 27.3% дисперсии.